## Colab Stuff

It's highly recommended to download the dataset first to your drive and work on Google Colab to save your time (and your local machine too:)))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/RL'

## Datasets

The official website of `PASCAL VOC` dataset is not working. So, you will need to search for a new way:
1. This is an alternative website which we used: [https://deepai.org/dataset/pascal-voc](https://deepai.org/dataset/pascal-voc)
2. Use our shared version from [Google Drive](https://drive.google.com/drive/folders/1wyB3XcD8EhiZI1BZYZ5SawG-DOwZzBV_?usp=sharing) in case the alternative website isn't available too.

P.S. Our drive folder contains only `PASCAL VOC 2007`. If you need a compressed folder for `PASCAL VOC 2012`, you can easily find it [here](https://drive.google.com/file/d/1XU7qwUVnV2S_iiWfb4DEmoxjbnQJxpHE/view?usp=sharing://) on our Google Drive too :)

## Reading Dataset

In [ ]:
%cd DRL-Object-Detection-main/

In [ ]:
from utils.agent import *
from utils.dataset import read_voc_dataset
from IPython.display import clear_output
import tqdm.notebook as tq
import pickle
%reload_ext autoreload
%autoreload 2

train_loader2007, val_loader2007 = read_voc_dataset(path="." ,year='2007')

## Dataset Organization

In [ ]:
#put datalist in list in case of combination of different datasets
#For example: [train_loader2007, train_loader2012]
datasets_per_class_train = sort_class_extract([train_loader2007])

In [ ]:
datasets_per_class_test = sort_class_extract([val_loader2007])

## Training Loop

In [ ]:

for i in tq.tqdm(range(len(classes))):
    classe = classes[i]
    print("Classe "+str(classe)+"...")
    #agent = Agent(classe, alpha=0.2, num_episodes=15, load=False, model_name='vgg16')
    agent = Agent_3alpha(classe, alpha=0.2, num_episodes=15, load=False, model_name='vgg16_3step')
    agent.train(datasets_per_class_train[classe])
    del agent
    torch.cuda.empty_cache()


**TESTING**

In [ ]:
torch.cuda.empty_cache()
results = {}
for i in classes:
    results[i] = []
model_name='vgg16'
for i in tq.tqdm(range(len(classes))):
    classe = classes[i]
    print("Class "+str(classe)+"...")
    agent = Agent(classe, load=True, model_name=model_name)
    res = agent.evaluate(datasets_per_class[classe])
    results[classe] = res

In [ ]:
file_name = 'classes_results_' + model_name + '.pickle'
with open(file_name, 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

**VISUALIZE**

In [ ]:
from utils.tools import classes
classe = random.choice(classes)
indices = np.random.choice(list(datasets_per_class[classe].keys()), size=5, replace=False)
agent = Agent(classe, load=True, model_name=model_name)

print("Class: " + classe)
for index in indices:
    image, gt_boxes = extract(index, datasets_per_class[classe])
    agent.predict_image(image, plot=True)